In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd

In [5]:
qdrant_client = QdrantClient(url="http://localhost:6333")

qdrant_client.create_collection(
    collection_name="mpv_dataset_farmaci",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `mpv_dataset_farmaci` already exists!"},"time":0.000238541}'

In [6]:
df_items = pd.read_json("/Users/enzomariasavelli/Documents/ai_bootcamp/project_1/aiengineeringbootcamp/data/output/mvp_dataset.json")

In [7]:
df_items.head(2)

,FDI_0001,FDI_0004,FDI_0008,FDI_0329,FDI_0339,FDI_0456,DENOMINAZIONE,ECCIPIENTI,DESCRIZIONE_GENERALE,CODICE_ICD,DESCRIZIONE_ICD,DESTINAZIONE_USO,PRINCIPIO_ATTIVO
0,367058,TISANA KELEMATA*OS POLV 80G,D,A06AB06,340800,1041,TISANA KEL&Egrave;MATA &ldquo;POLVERE PER TISA...,"Gramigna, guaiaco, issopo, parietaria, menta p...",Trattamento di breve durata della stitichezza ...,None,None,None,SENNA FOGLIE-100 g contengono: <b>Principio at...
1,367072,"TISANA KELEMATA*20BUST 1,3G",D,A06AB06,340800,1041,TISANA KELEMATA,"Gramigna, guaiaco, issopo, parietaria, menta p...",Trattamento di breve durata della stitichezza ...,None,None,None,SENNA FOGLIE-Ogni bustina contiene: <table bor...


In [8]:
def preprocess_data(row):
    return f"{row['FDI_0004']} {row['ECCIPIENTI']} {row['DESCRIZIONE_GENERALE']}{row['PRINCIPIO_ATTIVO']}{row['DESCRIZIONE_ICD']}"

In [9]:
df_items['preprocessed_data'] = df_items.apply(preprocess_data, axis=1)


In [10]:
df_items.head(2)

,FDI_0001,FDI_0004,FDI_0008,FDI_0329,FDI_0339,FDI_0456,DENOMINAZIONE,ECCIPIENTI,DESCRIZIONE_GENERALE,CODICE_ICD,DESCRIZIONE_ICD,DESTINAZIONE_USO,PRINCIPIO_ATTIVO,preprocessed_data
0,367058,TISANA KELEMATA*OS POLV 80G,D,A06AB06,340800,1041,TISANA KEL&Egrave;MATA &ldquo;POLVERE PER TISA...,"Gramigna, guaiaco, issopo, parietaria, menta p...",Trattamento di breve durata della stitichezza ...,None,None,None,SENNA FOGLIE-100 g contengono: <b>Principio at...,"TISANA KELEMATA*OS POLV 80G Gramigna, guaiaco,..."
1,367072,"TISANA KELEMATA*20BUST 1,3G",D,A06AB06,340800,1041,TISANA KELEMATA,"Gramigna, guaiaco, issopo, parietaria, menta p...",Trattamento di breve durata della stitichezza ...,None,None,None,SENNA FOGLIE-Ogni bustina contiene: <table bor...,"TISANA KELEMATA*20BUST 1,3G Gramigna, guaiaco,..."


In [11]:
import tiktoken

def num_tokens_from_string(string: str, model_name: str = "text-embedding-3-small") -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    return len(encoding.encode(string))

In [12]:
tokens = []
for i in range(len(df_items)):
    tokens.append(num_tokens_from_string(df_items['preprocessed_data'].iloc[i], "text-embedding-3-small"))


print(tokens[0:10])


[1037, 1168, 1016, 798, 898, 895, 753, 886, 1319, 1360]


In [13]:
index_allowed_tokens = [i for i in range(len(tokens)) if tokens[i] < 6000]
print(len(index_allowed_tokens))



187073


In [14]:
df_filtered = df_items.iloc[index_allowed_tokens]
print(len(df_filtered))

187073


In [15]:
df_sample = df_filtered.sample(n=50,random_state=55)

In [16]:
import openai

def get_embeddings(text, model_name="text-embedding-3-small"):
    response = openai.embeddings.create(
        input = [text],
        model = model_name
    )
    return response.data[0].embedding


In [17]:
data_to_embed = df_sample['preprocessed_data'].tolist()

pointstructs = []
for i,data in enumerate(data_to_embed):
    embedding = get_embeddings(data)
    pointstructs.append(PointStruct(id=i, vector=embedding, payload={"text": data}))


In [18]:
pointstructs 

[PointStruct(id=0, vector=[0.00851596612483263, 0.017380012199282646, 0.04412218555808067, 0.05161191523075104, -0.0366564616560936, -0.006865585222840309, -0.01575963757932186, -0.0033517738338559866, -0.016179734840989113, -0.05458860471844673, 0.015291530638933182, -0.03766469657421112, 0.0019114413298666477, 0.02071678265929222, 0.04815511777997017, -0.015615604817867279, 0.00832392182201147, -0.01769208535552025, 0.010034316219389439, 0.008203893899917603, 0.04582658037543297, -0.01808817684650421, -0.04181765392422676, -0.060686010867357254, -0.0006211433792486787, -0.0673595517873764, 0.008822036907076836, -0.044074174016714096, -0.007891821675002575, -0.030559055507183075, 0.014835424721240997, -0.04040132835507393, 0.01578364335000515, -0.029238751158118248, 0.04306594282388687, -0.007147650234401226, 0.00945818331092596, 0.009626222774386406, -0.044962380081415176, -0.05281219258904457, -0.013491114601492882, 0.00529022142291069, 0.030751099810004234, 0.0518999807536602, -0.0

In [19]:
qdrant_client.upsert(
    collection_name="mpv_dataset_farmaci",
    points=pointstructs
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
def retrieve_data(query):
    query_embedding = get_embeddings(query)
    results = qdrant_client.query_points(
        collection_name="mpv_dataset_farmaci",
        query=query_embedding,
        limit=10
    )
    return results

In [21]:
retrieve_data("QUale farmaco posso usare per il mal di testa?").points

[ScoredPoint(id=20, version=3, score=0.43459195, payload={'text': "PULVISED 30CPR None br> <b>Ingredienti</b><br> Griffonia semplicifolia estratto secco (titolato 93% in 5-idrossitriptofano) 80 mg; crataegus oxyacantha estratto secco 100 mg; escholtzia californica estratto secco 100 mg; passiflora incarnata estratto secco 150 mg; papaver rhoeas estratto secco 50 mg; vitamina B6 6 mg.<br> <br> <b>Modalit&agrave; d'uso</b><br> 1-3 compresse distribuite nell&rsquo;arco della giornata, a seconda della necessit&agrave;. Se necessario, in alternativa, si possono assumere 1-2 compresse mezz&rsquo;ora prima di coricarsi.<br> <br> <b>Avvertenze</b><br> Non superare le dosi consigliate. Gli integratori non vanno intesi come sostituti di una dieta variata. Il prodotto deve essere tenuto fuori dalla portata dei bambini al di sotto dei tre anni di et&agrave;.<br> <br> <b>Formato</b><br> 30 compresse.<br> <br>NESSUN PRINCIPIO ATTIVO-nanNone"}, vector=None, shard_key=None, order_value=None),
 ScoredP

In [22]:
qdrant_client = QdrantClient(url="http://localhost:6333")

qdrant_client.create_collection(
    collection_name="dataset_farmaci_2",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `dataset_farmaci_2` already exists!"},"time":0.0000665}'

In [ ]:
df_sample = df_items.sample(n=50, random_state=25)

In [23]:
def get_embeddings(text, model_name="text-embedding-3-small"):
    response = openai.embeddings.create(
        input = [text],
        model = model_name
    )
    return response.data[0].embedding

In [24]:
data_to_embed = df_sample['preprocessed_data'].tolist()

pointstructs = []
for i,data in enumerate(data_to_embed):
    embedding = get_embeddings(data)
    pointstructs.append(PointStruct(id=i, vector=embedding, payload={"text": data}))

In [25]:
qdrant_client.upsert(
    collection_name="dataset_farmaci_2",
    wait=True,
    points=pointstructs
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [26]:
import json

output_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Suggested question.",
            },
            "chunk_ids": {
                "type": "array",
                "items": {
                    "type": "integer",
                    "description": "Index of the chunk that could be used to answer the question.",
                },
            },
            "answer_example": {
                "type": "string",
                "description": "Suggested answer grounded in the contexr.",
            },
            "reasoning": {
                "type": "string",
                "description": "Reasoning why the question could be answered with the chunks.",
            },
        },
    },
}


SYSTEM_PROMPT = f"""
I am building a RAG application. I have a collection of 50 chunks of text. 
The RAG application will act as a pharmacist assistant who can answer questions about which medicines to use for a specific condition.
I will provide you with all of the available medicines with indexes of each chunk.
I want you to come up with 30 questions to which the answers will be grounded in the chunks context.
As an output I need you to provide me with a list of questions and the index of the chunk that could be used to answer the question.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose those chunks to answer the question.
Try to come up with a mix of questions that could use multiple chunks and questions that could use a single chunk.
Also, include 5 questions that can't be answered with the available chunks.

{json.dumps(output_schema, indent=2)}

I need to be able to parse the json output.
"""

USER_PROMPT = f"""
Here is the list of chunks. Each list element is a dictionary with id and text.
{[{"id": i, "text": data} for i, data in enumerate(data_to_embed)]}
"""

In [27]:
print(SYSTEM_PROMPT)


I am building a RAG application. I have a collection of 50 chunks of text. 
The RAG application will act as a pharmacist assistant who can answer questions about which medicines to use for a specific condition.
I will provide you with all of the available medicines with indexes of each chunk.
I want you to come up with 30 questions to which the answers will be grounded in the chunks context.
As an output I need you to provide me with a list of questions and the index of the chunk that could be used to answer the question.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose those chunks to answer the question.
Try to come up with a mix of questions that could use multiple chunks and questions that could use a single chunk.
Also, include 5 questions that can't be answered with the available chunks.

{
  "type": "array",
  "items": {
    "type": "object",
    "properties": {
      "question": {
        "type": "string",


In [28]:
print(USER_PROMPT)


Here is the list of chunks. Each list element is a dictionary with id and text.
[{'id': 0, 'text': 'CALCAREA CARBON 200CH GR 4G None br>Medicinale omeopatico senza  indicazioni terapeutiche approvateNESSUN PRINCIPIO ATTIVO-nanNone'}, {'id': 1, 'text': "AGLIO COMP 50CPS 29,5G None br> <font size=2><b>HEALTH</b></font><br> <br> <font size=5>Aglio<br> composto</font><br> <br> <font size=2>Integratore alimentare<br> a base di estratti vegetali</font><br> <br> <div align=justify>Il prodotto favorisce:<br> &bull; la regolare funzionalit&agrave; dell'apparato cardiovascolare;<br> &bull; la regolarit&agrave; della pressione arteriosa. <br> <br> <b>Ingredienti</b><br> Aglio bulbo (<i>Allium sativum</i>) estratto secco titolato all'1% alliina, Biancospino fiori e foglie (<i>Crataegus oxiacantha</i>) estratto secco titolato 1,5% vitexina, Ulivo foglie (<i>Olea europaea</i>) estratto secco titolato 6% oleuropeina.<br> Coadiuvanti: talco.<br> Capsule in gelatina vegetale.<br> Non contiene colorant

In [29]:
response = openai.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]
)

print(response.choices[0].message.content)

```json
[
  {
    "question": "Qual è il medicinale consigliato come coadiuvante per la funzionalità epatica?",
    "chunk_ids": [26],
    "answer_example": "Il prodotto FEGATO 250G è un integratore alimentare con carciofo, le cui foglie coadiuvano la funzione epatica.",
    "reasoning": "Il chunk 26 descrive un prodotto specifico per la funzione epatica a base di estratti di carciofo."
  },
  {
    "question": "Cosa è consigliato per favorire l'equilibrio del peso corporeo e il controllo del senso di fame?",
    "chunk_ids": [10, 46],
    "answer_example": "Entrambi Kiloburn Forte e Inverter Control P sono utili per l’equilibrio del peso corporeo e il controllo del senso di fame, grazie a ingredienti come opuntia, garcinia cambogia e griffonia.",
    "reasoning": "Entrambi i chunk 10 e 46 descrivono integratori utili per il controllo del peso e della fame."
  },
  {
    "question": "Quale prodotto aiuta a ridurre la pressione intraoculare nei pazienti con glaucoma?",
    "chunk_ids": 

In [31]:
import json
json_output = response.choices[0].message.content
json_output = json_output.replace("```json", "")
json_output = json_output.replace("```", "")
json_output = json.loads(json_output)






In [32]:
json_output

[{'question': 'Qual è il medicinale consigliato come coadiuvante per la funzionalità epatica?',
  'chunk_ids': [26],
  'answer_example': 'Il prodotto FEGATO 250G è un integratore alimentare con carciofo, le cui foglie coadiuvano la funzione epatica.',
  'reasoning': 'Il chunk 26 descrive un prodotto specifico per la funzione epatica a base di estratti di carciofo.'},
 {'question': "Cosa è consigliato per favorire l'equilibrio del peso corporeo e il controllo del senso di fame?",
  'chunk_ids': [10, 46],
  'answer_example': 'Entrambi Kiloburn Forte e Inverter Control P sono utili per l’equilibrio del peso corporeo e il controllo del senso di fame, grazie a ingredienti come opuntia, garcinia cambogia e griffonia.',
  'reasoning': 'Entrambi i chunk 10 e 46 descrivono integratori utili per il controllo del peso e della fame.'},
 {'question': 'Quale prodotto aiuta a ridurre la pressione intraoculare nei pazienti con glaucoma?',
  'chunk_ids': [3],
  'answer_example': 'XALOST*COLL è indicato

In [40]:
from langsmith import Client
import os 
from dotenv import load_dotenv

load_dotenv()

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

dataset_name = "rag_evaluation_dataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for evaluating the RAG pipeline"
)

In [41]:
for item in json_output:
    client.create_example(
        dataset_id=dataset.id,
        inputs={"question": item['question']},
        outputs={
            "grounded_answer": item['answer_example'],
            "context_ids": item['chunk_ids'],
            "context": [qdrant_client.retrieve(collection_name="dataset_farmaci_2", ids=[id], with_payload=True)[0].payload['text'] for id in item['chunk_ids']]
        }
    )